In [1]:
!python -m jupytools export -nb "00e_prepare_channels_stats.ipynb" -o .

Exported: 00e_prepare_channels_stats.ipynb -> prepare_channels_stats.py
1 notebook(s) exported into folder: .


In [1]:
#export
import json
import os
from os.path import dirname, join
from pdb import set_trace

import jupytools
import jupytools.syspath
import pandas as pd

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
import contextlib, io
with contextlib.redirect_stderr(io.StringIO()):
    from basedir import ROOT
    from dataset import load_data
    from visual import rgb, six

In [2]:
#export
data_dict = load_data()
train_records, _ = data_dict['data']

In [3]:
train_records[:1]

[{'id_code': 'HEPG2-01_1_B03',
  'site': 1,
  'channel': 1,
  'cell_type': 'HEPG2',
  'experiment': 'HEPG2-01',
  'plate': 1,
  'sirna': 513,
  'well': 'B03',
  'well_type': 'treatment',
  'images': [[1, '/home/ck/data/protein/train/HEPG2-01/Plate1/B03_s1_w1.png'],
   [2, '/home/ck/data/protein/train/HEPG2-01/Plate1/B03_s1_w2.png'],
   [3, '/home/ck/data/protein/train/HEPG2-01/Plate1/B03_s1_w3.png'],
   [4, '/home/ck/data/protein/train/HEPG2-01/Plate1/B03_s1_w4.png'],
   [5, '/home/ck/data/protein/train/HEPG2-01/Plate1/B03_s1_w5.png'],
   [6, '/home/ck/data/protein/train/HEPG2-01/Plate1/B03_s1_w6.png']],
  'enc_sirna': 513}]

In [4]:
#export
stats = pd.read_csv(ROOT/'pixel_stats.csv')

In [5]:
stats.head(6)

,id_code,experiment,plate,well,site,channel,mean,std,median,min,max
0,HEPG2-01_1_B02,HEPG2-01,1,B02,1,1,71.063782,43.146240,67.0,7,255
1,HEPG2-01_1_B02,HEPG2-01,1,B02,1,2,32.174431,9.384594,31.0,6,98
2,HEPG2-01_1_B02,HEPG2-01,1,B02,1,3,61.836025,23.377997,59.0,11,255
3,HEPG2-01_1_B02,HEPG2-01,1,B02,1,4,56.983257,16.011435,56.0,11,156
4,HEPG2-01_1_B02,HEPG2-01,1,B02,1,5,91.671993,39.221836,85.0,13,255
5,HEPG2-01_1_B02,HEPG2-01,1,B02,1,6,49.634388,15.679836,48.0,10,255


In [ ]:
#export
channels = []
for (id_code, site), g in tdqm(stats.groupby(['id_code', 'site'])):
    g = g.sort_values(by='channel')
    channel_stats = g[['channel', 'mean', 'std', 'min', 'max']].to_dict('records')
    for d in channel_stats:
        d['channel'] = int(d['channel'])
        d['mean'], d['std'] = float(d['mean']), float(d['std'])
        d['min'], d['max'] = int(d['min']), int(d['max'])
    channels.append({'id_code': id_code, 'site': int(site), 'stats': channel_stats})

In [ ]:
#export
with open('channels.json', 'w') as f:
    json.dump(channels, f)